In [ ]:
import gradio as gr
import os
import tempfile
import shutil

def process_pdfs(pdf_files):
    # Create a temporary directory
    tmp_dir = tempfile.mkdtemp()
    print("tmp_dir", tmp_dir)
    # Save uploaded PDFs to the temporary directory
    saved_files = []
    for pdf_file in pdf_files:
        # Construct the destination file path
        dest_path = os.path.join(tmp_dir, os.path.basename(pdf_file))
        # Move the uploaded file to the temporary directory
        shutil.copy(pdf_file, dest_path)
        saved_files.append(dest_path)  # Store the path of the saved file

    # For demonstration, return the paths of saved files
    return saved_files

# Create the Gradio interface
iface = gr.Interface(
    fn=process_pdfs, 
    inputs=gr.File(label="Upload PDFs", type="filepath", file_count="multiple"),  # Allow multiple file uploads
    outputs="text",
    title="Multiple PDF Uploader",
    description="Upload multiple PDF files to process."
)

# Launch the interface
iface.launch()
import gradio as gr
import os
import tempfile
import shutil

def process_pdfs(pdf_files):
    # Create a temporary directory
    tmp_dir = tempfile.mkdtemp()
    print("tmp_dir", tmp_dir)
    # Save uploaded PDFs to the temporary directory
    saved_files = []
    for pdf_file in pdf_files:
        # Construct the destination file path
        dest_path = os.path.join(tmp_dir, os.path.basename(pdf_file))
        # Move the uploaded file to the temporary directory
        shutil.copy(pdf_file, dest_path)
        saved_files.append(dest_path)  # Store the path of the saved file

    # For demonstration, return the paths of saved files
    return saved_files

# Create the Gradio interface
iface = gr.Interface(
    fn=process_pdfs, 
    inputs=gr.File(label="Upload PDFs", type="filepath", file_count="multiple"),  # Allow multiple file uploads
    outputs="text",
    title="Multiple PDF Uploader",
    description="Upload multiple PDF files to process."
)

# Launch the interface
iface.launch()


In [ ]:
! pip install gradio llama_index langchain_community langchain langchain_huggingface llama_index.vector_stores.opensearch llama-index-embeddings-langchain langchain-ollama langchain_huggingface

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from llama_index.vector_stores.opensearch import OpensearchVectorStore, OpensearchVectorClient
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core import VectorStoreIndex, StorageContext
from opensearchpy import OpenSearch

#3
    # http endpoint for your cluster (opensearch required for vector index usage)
endpoint = "http://localhost:9200"
    # index to demonstrate the VectorStore impl
idx =  "test_pdf_index"

index_name = "gradio-brochures-bge-m3-1024"
auth = ('admin', 'LocalIndex@42535') 

    # Initialize OpenSearch client
client = OpenSearch(
            hosts=["https://localhost:9200/"],
            http_compress=True,
            use_ssl=True,  # DONT USE IN PRODUCTION
            verify_certs=False,  # DONT USE IN PRODUCTION
            ssl_assert_hostname=False,
            http_auth=auth,
            ssl_show_warn=False,
        )

# 2. Setup HuggingFace Embeddings using HuggingFaceEmbeddings from LangChain
#embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# 3. Connect to an Existing OpenSearch Index with OpenSearchVectorStore
#index_name = "your_existing_index"

model_name = "BAAI/bge-m3"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
hf = HuggingFaceEmbeddings(
        model_name=model_name,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )
# OpensearchVectorClient stores text in this field by default
text_field = "parsed_pdf_content"
# OpensearchVectorClient stores embeddings in this field by default
embedding_field = "embedding"

# single opensearch index with vector search enabled with hybrid search pipeline
client = OpensearchVectorClient(
        endpoint=endpoint,
        index=index_name,
        embedding_field=embedding_field,
        text_field=text_field,
        #http_auth=auth,
        #use_ssl = True,
        dim=1024,
        #verify_certs = False,
        #ssl_assert_hostname = False,
        #ssl_show_warn = False,
        search_pipeline="hybrid-search-pipeline",
        os_client=client
        #settings=settings
    )
vector_store = OpensearchVectorStore(client)

#storage_context = StorageContext.from_defaults(vector_store=vector_store)

vectorStoreIndex = VectorStoreIndex(storage_context=storage_context, embed_model=hf)

# 4. Create the Vector Index
#index = VectorIndex(vector_store=vector_store)
retriever = VectorIndexRetriever(
    index=vectorStoreIndex,
    similarity_top_k=10,
)




# 5. Query for Similar Documents
query = "Tell me about london"

# Perform the query
response = retriever.query(query)

# 6. Output the search results
for result in response:
    print(f"Document: {result.text}\nScore: {result.score}")
#

In [ ]:
#https://www.cianclarke.com/blog/aws-opensearch-and-langchain/

from langchain.vectorstores import OpenSearchVectorSearch
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from llama_index.vector_stores.opensearch import OpensearchVectorStore, OpensearchVectorClient
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core import VectorStoreIndex, StorageContext
from opensearchpy import OpenSearch
# create HuggingFaceEmbeddings with BGE embeddings

dimensions = 1024

model_name = "BAAI/bge-m3"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
hf = HuggingFaceEmbeddings(
        model_name=model_name,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )
index_name = "gradio-brochures-bge-m3-1024"
auth = ('admin', 'LocalIndex@42535') 
docsearch = OpenSearchVectorSearch(
    embedding_function=hf,
    opensearch_url=f'https://localhost:9200/',
    http_compress=True,
    use_ssl=True,  # DONT USE IN PRODUCTION
    verify_certs=False,  # DONT USE IN PRODUCTION
    ssl_assert_hostname=False,
    http_auth=auth,
    ssl_show_warn=False,
    index_name=index_name
)

# Document Search
query = "Tell me about London"

docs = docsearch.similarity_search(query, k=2, vector_field="embedding", text_field="parsed_pdf_content")

print('Total results:', len(docs))
# The result here should be the document which closest resembles our question - the RAG phase actually formats an answer. 
print('Best result:', docs[0].page_content)

In [1]:
import json

# This is the original data structure
data_list = [
    {
        "id": 1,
        "name": "Apple",
        "details_json": '{"color": "Red", "price": 1.99, "is_fresh": true}'
    },
    {
        "id": 2,
        "name": "Banana",
        "details_json": '{"color": "Yellow", "price": 0.79, "origin": "Ecuador"}'
    },
    {
        "id": 3,
        "name": "Grape",
        "details_json": '{"color": "Purple", "price": 2.49, "attributes": ["seedless", "sweet"]}'
    }
]

print("Original Data Structure:")
print(data_list)
print(f"Type of data_list: {type(data_list)}")
print(f"Type of the embedded JSON string: {type(data_list[0]['details_json'])}")

# To access the data within the embedded JSON, you need to parse it
# For example, accessing the 'color' of the first item
first_item_details_string = data_list[0]['details_json']
first_item_details_dict = json.loads(first_item_details_string)

print("\nAccessing Embedded JSON Data:")
print(f"Parsed details for the first item: {first_item_details_dict}")
print(f"Type after json.loads(): {type(first_item_details_dict)}")
print(f"Color of the first item: {first_item_details_dict['color']}")


Original Data Structure:
[{'id': 1, 'name': 'Apple', 'details_json': '{"color": "Red", "price": 1.99, "is_fresh": true}'}, {'id': 2, 'name': 'Banana', 'details_json': '{"color": "Yellow", "price": 0.79, "origin": "Ecuador"}'}, {'id': 3, 'name': 'Grape', 'details_json': '{"color": "Purple", "price": 2.49, "attributes": ["seedless", "sweet"]}'}]
Type of data_list: <class 'list'>
Type of the embedded JSON string: <class 'str'>

Accessing Embedded JSON Data:
Parsed details for the first item: {'color': 'Red', 'price': 1.99, 'is_fresh': True}
Type after json.loads(): <class 'dict'>
Color of the first item: Red


In [6]:
import json

# This is the original data structure
data_list = [
    {
        "rule": "fnd test",
        "sql": "BAT.FIND_TEST_IND = TRUE",
        "category": "FIND_TEST_IND",
        "bat_col_meta_info": "### FIND_TEST_IND\n**Attribute Name**: 'FIND_TEST_IND' \n**Data Types**: BOOLEAN\n**Description**: A bool field that indicates whether or not an account was part of Find Test. If 'Null', then the account was not part of a FT. "
    }
]

print("Original Data Structure:")
print(data_list)
print(f"Type of data_list: {type(data_list)}")
print(f"Type of the embedded JSON string: {type(data_list[0]['bat_col_meta_info'])}")

# To access the data within the embedded JSON, you need to parse it
# For example, accessing the 'color' of the first item
first_item_details_string = data_list[0]['bat_col_meta_info']
first_item_details_dict = json.loads(first_item_details_string)

print("\nAccessing Embedded JSON Data:")
print(f"Parsed details for the first item: {first_item_details_dict}")
print(f"Type after json.loads(): {type(first_item_details_dict)}")
print(f"Color of the first item: {first_item_details_dict['FIND_TEST_IND']}")


Original Data Structure:
[{'rule': 'fnd test', 'sql': 'BAT.FIND_TEST_IND = TRUE', 'category': 'FIND_TEST_IND', 'bat_col_meta_info': "### FIND_TEST_IND\n**Attribute Name**: 'FIND_TEST_IND' \n**Data Types**: BOOLEAN\n**Description**: A bool field that indicates whether or not an account was part of Find Test. If 'Null', then the account was not part of a FT. "}]
Type of data_list: <class 'list'>
Type of the embedded JSON string: <class 'str'>


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [7]:
import json
import re
from typing import Any, Dict, List, Union, Tuple
from datetime import datetime, date
from decimal import Decimal

# Sample data_list with various problematic types
data_list = [
    {
        "rule": "fnd test",
        "sql": "BAT.FIND_TEST_IND = TRUE",
        "category": "FIND_TEST_IND",
        "bat_col_meta_info": "### FIND_TEST_IND\n**Attribute Name**: 'FIND_TEST_IND' \n**Data Types**: BOOLEAN\n**Description**: A bool field that indicates whether or not an account was part of Find Test. If 'Null', then the account was not part of a FT. "
    },
    {
        "id": 123,
        "name": "Test User",
        "active": True,
        "score": 95.5
    },
    {
        "timestamp": "2024-01-15T10:30:00",
        "data": '{"nested": "json", "value": 42}',
        "tags": ["tag1", "tag2"],
        "metadata": {"key": "value"}
    },
    {
        "decimal_val": "123.456",
        "bool_str": "true",
        "null_str": "null",
        "empty": "",
        "complex": "### Header\n**Field**: Value"
    }
]


class JSONValidator:
    """
    A comprehensive JSON validator and converter for Dict[str, Any] structures.
    """
    
    def __init__(self):
        self.validation_report = {
            "total_items": 0,
            "total_fields": 0,
            "invalid_fields": [],
            "conversions": [],
            "errors": []
        }
    
    def identify_value_type(self, value: Any) -> Dict[str, Any]:
        """
        Identify the type and characteristics of a value.
        """
        result = {
            "original_value": value,
            "original_type": type(value).__name__,
            "is_json_native": False,
            "needs_conversion": False,
            "detected_format": None,
            "error": None
        }
        
        # Check if it's a JSON native type (str, int, float, bool, None, list, dict)
        json_native_types = (str, int, float, bool, type(None), list, dict)
        
        if isinstance(value, json_native_types):
            result["is_json_native"] = True
            
            # Even if native type, check if it needs special handling
            if isinstance(value, str):
                result["detected_format"] = self._classify_string(value)
                if result["detected_format"] in ["embedded_json", "markdown", "numeric_string", "boolean_string"]:
                    result["needs_conversion"] = True
            elif isinstance(value, (list, dict)):
                # Check nested structures
                result["needs_conversion"] = self._needs_nested_conversion(value)
        else:
            result["is_json_native"] = False
            result["needs_conversion"] = True
            result["detected_format"] = "non_json_type"
        
        # Test JSON serializability
        try:
            json.dumps(value)
        except (TypeError, ValueError, OverflowError) as e:
            result["needs_conversion"] = True
            result["error"] = str(e)
        
        return result
    
    def _classify_string(self, text: str) -> str:
        """
        Classify the content type of a string.
        """
        if not isinstance(text, str):
            return "non_string"
        
        text_stripped = text.strip()
        
        if not text_stripped:
            return "empty_string"
        
        # Check for embedded JSON
        if text_stripped.startswith(('{', '[')):
            try:
                json.loads(text_stripped)
                return "embedded_json"
            except:
                pass
        
        # Check for markdown
        if re.search(r'###\s+\w+|\*\*.+?\*\*', text):
            return "markdown"
        
        # Check for XML
        if text_stripped.startswith('<') and text_stripped.endswith('>'):
            return "xml"
        
        # Check for ISO datetime
        iso_date_pattern = r'\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}'
        if re.match(iso_date_pattern, text_stripped):
            return "iso_datetime"
        
        # Check for date
        date_patterns = [
            r'^\d{4}-\d{2}-\d{2}$',
            r'^\d{2}/\d{2}/\d{4}$',
            r'^\d{2}-\d{2}-\d{4}$'
        ]
        for pattern in date_patterns:
            if re.match(pattern, text_stripped):
                return "date_string"
        
        # Check for numeric string
        try:
            float(text_stripped)
            return "numeric_string"
        except ValueError:
            pass
        
        # Check for boolean string
        if text_stripped.lower() in ['true', 'false', 'yes', 'no', '1', '0']:
            return "boolean_string"
        
        # Check for null string
        if text_stripped.lower() in ['null', 'none', 'nil']:
            return "null_string"
        
        return "plain_text"
    
    def _needs_nested_conversion(self, value: Union[list, dict]) -> bool:
        """
        Check if nested structures need conversion.
        """
        try:
            if isinstance(value, dict):
                for v in value.values():
                    info = self.identify_value_type(v)
                    if info["needs_conversion"]:
                        return True
            elif isinstance(value, list):
                for item in value:
                    info = self.identify_value_type(item)
                    if info["needs_conversion"]:
                        return True
            return False
        except:
            return True
    
    def parse_markdown_to_dict(self, markdown_text: str) -> Dict[str, str]:
        """
        Parse markdown formatted text into a structured dictionary.
        """
        result = {"_format": "parsed_markdown"}
        
        # Extract header
        header_match = re.search(r'###\s+(.+?)(?=\n|$)', markdown_text)
        if header_match:
            result['header'] = header_match.group(1).strip()
        
        # Extract key-value pairs with bold keys
        pattern = r'\*\*(.+?)\*\*:\s*(.+?)(?=\n\*\*|\n###|$)'
        matches = re.findall(pattern, markdown_text, re.DOTALL)
        
        for key, value in matches:
            clean_key = key.strip().lower().replace(' ', '_')
            clean_value = value.strip().strip("'\"")
            result[clean_key] = clean_value
        
        # If no structured data found, return original
        if len(result) == 1:  # Only has _format key
            result['raw_content'] = markdown_text
        
        return result
    
    def convert_value(self, value: Any, type_info: Dict[str, Any]) -> Tuple[Any, str]:
        """
        Convert a value to JSON-compatible format.
        Returns: (converted_value, conversion_description)
        """
        if not type_info["needs_conversion"]:
            return value, "no_conversion_needed"
        
        original_type = type_info["original_type"]
        detected_format = type_info["detected_format"]
        
        try:
            # Handle non-JSON native types
            if not type_info["is_json_native"]:
                if isinstance(value, (datetime, date)):
                    return value.isoformat(), f"datetime_to_iso_string"
                elif isinstance(value, Decimal):
                    return float(value), f"decimal_to_float"
                elif isinstance(value, bytes):
                    return value.decode('utf-8'), f"bytes_to_string"
                elif isinstance(value, set):
                    return list(value), f"set_to_list"
                elif isinstance(value, tuple):
                    return list(value), f"tuple_to_list"
                elif hasattr(value, '__dict__'):
                    return vars(value), f"object_to_dict"
                else:
                    return str(value), f"custom_type_to_string"
            
            # Handle string formats
            if isinstance(value, str):
                if detected_format == "embedded_json":
                    try:
                        parsed = json.loads(value)
                        return parsed, "parsed_embedded_json"
                    except:
                        return value, "embedded_json_parse_failed"
                
                elif detected_format == "markdown":
                    parsed = self.parse_markdown_to_dict(value)
                    return parsed, "parsed_markdown_to_dict"
                
                elif detected_format == "numeric_string":
                    # Keep as string for JSON, but note it could be converted
                    return value, "kept_as_numeric_string"
                
                elif detected_format == "boolean_string":
                    # Keep as string for JSON, but note it could be converted
                    return value, "kept_as_boolean_string"
                
                elif detected_format == "null_string":
                    return None, "null_string_to_null"
            
            # Handle nested structures
            if isinstance(value, dict):
                converted = {}
                for k, v in value.items():
                    v_info = self.identify_value_type(v)
                    converted[k], _ = self.convert_value(v, v_info)
                return converted, "converted_nested_dict"
            
            elif isinstance(value, list):
                converted = []
                for item in value:
                    item_info = self.identify_value_type(item)
                    converted_item, _ = self.convert_value(item, item_info)
                    converted.append(converted_item)
                return converted, "converted_nested_list"
            
            return value, "no_conversion_applied"
        
        except Exception as e:
            # If conversion fails, return string representation
            return str(value), f"conversion_error_fallback: {str(e)}"
    
    def validate_and_convert_dict(self, data_dict: Dict[str, Any], item_index: int) -> Dict[str, Any]:
        """
        Validate and convert a single dictionary.
        """
        converted_dict = {}
        
        for key, value in data_dict.items():
            self.validation_report["total_fields"] += 1
            
            # Identify value type
            type_info = self.identify_value_type(value)
            
            # Convert if needed
            if type_info["needs_conversion"]:
                converted_value, conversion_desc = self.convert_value(value, type_info)
                converted_dict[key] = converted_value
                
                # Log the conversion
                self.validation_report["invalid_fields"].append({
                    "item_index": item_index,
                    "field": key,
                    "original_type": type_info["original_type"],
                    "detected_format": type_info["detected_format"],
                    "error": type_info["error"]
                })
                
                self.validation_report["conversions"].append({
                    "item_index": item_index,
                    "field": key,
                    "conversion": conversion_desc,
                    "from_type": type(value).__name__,
                    "to_type": type(converted_value).__name__
                })
            else:
                converted_dict[key] = value
        
        return converted_dict
    
    def validate_and_convert_list(self, data_list: List[Dict[str, Any]]) -> Tuple[List[Dict[str, Any]], Dict]:
        """
        Validate and convert entire list of dictionaries.
        Returns: (converted_list, validation_report)
        """
        self.validation_report["total_items"] = len(data_list)
        converted_list = []
        
        for idx, item in enumerate(data_list):
            if not isinstance(item, dict):
                self.validation_report["errors"].append({
                    "item_index": idx,
                    "error": f"Item is not a dictionary, it's a {type(item).__name__}"
                })
                # Try to convert to dict or skip
                converted_list.append({"error": f"Invalid item type: {type(item).__name__}", "original": str(item)})
                continue
            
            try:
                converted_dict = self.validate_and_convert_dict(item, idx)
                converted_list.append(converted_dict)
            except Exception as e:
                self.validation_report["errors"].append({
                    "item_index": idx,
                    "error": str(e)
                })
                converted_list.append({"error": str(e), "original_keys": list(item.keys())})
        
        return converted_list, self.validation_report
    
    def generate_json(self, data_list: List[Dict[str, Any]], indent: int = 2) -> str:
        """
        Main method: Validate, convert, and generate valid JSON.
        """
        # Convert the data
        converted_list, report = self.validate_and_convert_list(data_list)
        
        # Generate JSON
        try:
            json_output = json.dumps(converted_list, indent=indent, ensure_ascii=False)
            
            # Validate by parsing back
            json.loads(json_output)
            
            return json_output
        except Exception as e:
            raise ValueError(f"Failed to generate valid JSON: {str(e)}")


# Usage
def main():
    print("=" * 100)
    print("ORIGINAL DATA LIST:")
    print("=" * 100)
    for idx, item in enumerate(data_list):
        print(f"\n--- Item {idx} ---")
        for key, value in item.items():
            print(f"  {key}: {repr(value)[:100]}...")
    
    # Create validator
    validator = JSONValidator()
    
    print("\n" + "=" * 100)
    print("VALIDATION AND CONVERSION ANALYSIS:")
    print("=" * 100)
    
    # Process each item
    for idx, item in enumerate(data_list):
        print(f"\n--- Item {idx} ---")
        for key, value in item.items():
            type_info = validator.identify_value_type(value)
            print(f"\n  Field: '{key}'")
            print(f"    Original Type: {type_info['original_type']}")
            print(f"    JSON Native: {type_info['is_json_native']}")
            print(f"    Needs Conversion: {type_info['needs_conversion']}")
            if type_info['detected_format']:
                print(f"    Detected Format: {type_info['detected_format']}")
            if type_info['error']:
                print(f"    Error: {type_info['error']}")
    
    print("\n" + "=" * 100)
    print("GENERATING VALID JSON:")
    print("=" * 100)
    
    try:
        json_output = validator.generate_json(data_list)
        print(json_output)
        
        print("\n" + "=" * 100)
        print("VALIDATION REPORT:")
        print("=" * 100)
        report = validator.validation_report
        
        print(f"\nTotal Items: {report['total_items']}")
        print(f"Total Fields: {report['total_fields']}")
        print(f"Invalid Fields Found: {len(report['invalid_fields'])}")
        print(f"Conversions Performed: {len(report['conversions'])}")
        print(f"Errors: {len(report['errors'])}")
        
        if report['invalid_fields']:
            print("\n--- Invalid Fields Details ---")
            for field_info in report['invalid_fields']:
                print(f"  Item {field_info['item_index']}, Field '{field_info['field']}':")
                print(f"    Type: {field_info['original_type']}")
                print(f"    Format: {field_info['detected_format']}")
                if field_info['error']:
                    print(f"    Error: {field_info['error']}")
        
        if report['conversions']:
            print("\n--- Conversions Performed ---")
            for conv in report['conversions']:
                print(f"  Item {conv['item_index']}, Field '{conv['field']}':")
                print(f"    Conversion: {conv['conversion']}")
                print(f"    {conv['from_type']} -> {conv['to_type']}")
        
        if report['errors']:
            print("\n--- Errors ---")
            for error in report['errors']:
                print(f"  Item {error['item_index']}: {error['error']}")
        
        print("\n" + "=" * 100)
        print("VERIFICATION:")
        print("=" * 100)
        
        # Verify JSON is valid
        parsed_back = json.loads(json_output)
        print(f"✓ Valid JSON generated successfully!")
        print(f"✓ Successfully parsed back from JSON")
        print(f"✓ Number of items: {len(parsed_back)}")
        print(f"✓ All items are valid JSON objects")
        
    except Exception as e:
        print(f"✗ Error generating JSON: {e}")
        import traceback
        traceback.print_exc()


if __name__ == "__main__":
    main()

ORIGINAL DATA LIST:

--- Item 0 ---
  rule: 'fnd test'...
  sql: 'BAT.FIND_TEST_IND = TRUE'...
  category: 'FIND_TEST_IND'...
  bat_col_meta_info: "### FIND_TEST_IND\n**Attribute Name**: 'FIND_TEST_IND' \n**Data Types**: BOOLEAN\n**Description**: ...

--- Item 1 ---
  id: 123...
  name: 'Test User'...
  active: True...
  score: 95.5...

--- Item 2 ---
  timestamp: '2024-01-15T10:30:00'...
  data: '{"nested": "json", "value": 42}'...
  tags: ['tag1', 'tag2']...
  metadata: {'key': 'value'}...

--- Item 3 ---
  decimal_val: '123.456'...
  bool_str: 'true'...
  null_str: 'null'...
  empty: ''...
  complex: '### Header\n**Field**: Value'...

VALIDATION AND CONVERSION ANALYSIS:

--- Item 0 ---

  Field: 'rule'
    Original Type: str
    JSON Native: True
    Needs Conversion: False
    Detected Format: plain_text

  Field: 'sql'
    Original Type: str
    JSON Native: True
    Needs Conversion: False
    Detected Format: plain_text

  Field: 'category'
    Original Type: str
    JSON Native

In [9]:
from typing import List, Dict, Any, Optional, Union
from datetime import datetime
from langchain_core.documents import Document
from pydantic import BaseModel, Field
import json


class ChunkMetadata(BaseModel):
    """Metadata for each document chunk"""
    documentId: str
    documentUrl: str
    documentVersion: str
    chunkSequence: int
    chunkTotal: int
    lastModified: str = Field(default_factory=lambda: datetime.utcnow().isoformat())


class CustomChunk(BaseModel):
    """Individual chunk with content and metadata"""
    chunkTokens: str
    metadata: ChunkMetadata
    
    def to_langchain(self) -> Document:
        """Convert to LangChain Document"""
        return Document(
            page_content=self.chunkTokens,
            metadata=self.metadata.model_dump()
        )
    
    @classmethod
    def from_langchain(cls, doc: Document) -> 'CustomChunk':
        """Create from LangChain Document"""
        return cls(
            chunkTokens=doc.page_content,
            metadata=ChunkMetadata(**doc.metadata)
        )


class CustomDocumentCollection(BaseModel):
    """Collection of document chunks"""
    items: List[CustomChunk]
    
    def to_json(self, filepath: Optional[str] = None, **kwargs) -> str:
        """Export to JSON format"""
        json_str = self.model_dump_json(indent=2, **kwargs)
        if filepath:
            with open(filepath, 'w') as f:
                f.write(json_str)
        return json_str
    
    def to_dict(self) -> Dict[str, Any]:
        """Export to dictionary format"""
        return self.model_dump()
    
    @classmethod
    def from_json(cls, json_data: str) -> 'CustomDocumentCollection':
        """Load from JSON string or file"""
        if json_data.endswith('.json'):
            with open(json_data, 'r') as f:
                json_data = f.read()
        return cls.model_validate_json(json_data)
    
    def to_langchain_docs(self) -> List[Document]:
        """Convert all chunks to LangChain Documents"""
        return [chunk.to_langchain() for chunk in self.items]
    
    @classmethod
    def from_langchain_docs(cls, docs: List[Document], 
                           document_id: str,
                           document_url: str,
                           document_version: str = "1.0") -> 'CustomDocumentCollection':
        """Create from list of LangChain Documents"""
        total_chunks = len(docs)
        chunks = []
        
        for idx, doc in enumerate(docs, start=1):
            metadata = ChunkMetadata(
                documentId=document_id,
                documentUrl=document_url,
                documentVersion=document_version,
                chunkSequence=idx,
                chunkTotal=total_chunks
            )
            chunks.append(CustomChunk(
                chunkTokens=doc.page_content,
                metadata=metadata
            ))
        
        return cls(items=chunks)


def dict_to_text(data: Dict[str, Any], indent: int = 0) -> str:
    """Convert dictionary to formatted text representation"""
    lines = []
    prefix = "  " * indent
    
    for key, value in data.items():
        if isinstance(value, dict):
            lines.append(f"{prefix}{key}:")
            lines.append(dict_to_text(value, indent + 1))
        elif isinstance(value, list):
            lines.append(f"{prefix}{key}: {json.dumps(value)}")
        elif isinstance(value, str) and ('\n' in value or len(value) > 100):
            # Handle multiline strings
            lines.append(f"{prefix}{key}:")
            for line in value.split('\n'):
                lines.append(f"{prefix}  {line}")
        else:
            lines.append(f"{prefix}{key}: {value}")
    
    return '\n'.join(lines)


def create_document_from_list(
    data_list: List[Dict[str, Any]],
    document_id: str,
    document_url: str,
    document_version: str = "1.0",
    chunk_by: str = "item"  # "item" or "size"
) -> CustomDocumentCollection:
    """
    Create document collection from a list of dictionaries
    
    Args:
        data_list: List of dictionaries to convert
        document_id: Document identifier
        document_url: Document URL/path
        document_version: Version string
        chunk_by: "item" (one chunk per item) or "size" (split by character count)
    """
    chunks = []
    
    if chunk_by == "item":
        # Create one chunk per list item
        total_chunks = len(data_list)
        
        for idx, item in enumerate(data_list, start=1):
            # Convert dict to readable text format
            chunk_text = dict_to_text(item)
            
            metadata = ChunkMetadata(
                documentId=document_id,
                documentUrl=document_url,
                documentVersion=document_version,
                chunkSequence=idx,
                chunkTotal=total_chunks
            )
            
            chunks.append(CustomChunk(
                chunkTokens=chunk_text,
                metadata=metadata
            ))
    
    return CustomDocumentCollection(items=chunks)


def create_document_from_list_json(
    data_list: List[Dict[str, Any]],
    document_id: str,
    document_url: str,
    document_version: str = "1.0",
    preserve_json: bool = True
) -> CustomDocumentCollection:
    """
    Create document collection preserving JSON structure
    
    Args:
        data_list: List of dictionaries to convert
        document_id: Document identifier
        document_url: Document URL/path
        document_version: Version string
        preserve_json: If True, store as JSON string; if False, use formatted text
    """
    chunks = []
    total_chunks = len(data_list)
    
    for idx, item in enumerate(data_list, start=1):
        if preserve_json:
            # Store as JSON string
            chunk_text = json.dumps(item, indent=2, ensure_ascii=False)
        else:
            # Store as formatted text
            chunk_text = dict_to_text(item)
        
        metadata = ChunkMetadata(
            documentId=document_id,
            documentUrl=document_url,
            documentVersion=document_version,
            chunkSequence=idx,
            chunkTotal=total_chunks
        )
        
        chunks.append(CustomChunk(
            chunkTokens=chunk_text,
            metadata=metadata
        ))
    
    return CustomDocumentCollection(items=chunks)


# Test with your sample data
if __name__ == "__main__":
    data_list = [
        {
            "rule": "fnd test",
            "sql": "BAT.FIND_TEST_IND = TRUE",
            "category": "FIND_TEST_IND",
            "bat_col_meta_info": "### FIND_TEST_IND\n**Attribute Name**: 'FIND_TEST_IND' \n**Data Types**: BOOLEAN\n**Description**: A bool field that indicates whether or not an account was part of Find Test. If 'Null', then the account was not part of a FT. "
        },
        {
            "id": 123,
            "name": "Test User",
            "active": True,
            "score": 95.5
        },
        {
            "timestamp": "2024-01-15T10:30:00",
            "data": '{"nested": "json", "value": 42}',
            "tags": ["tag1", "tag2"],
            "metadata": {"key": "value"}
        },
        {
            "decimal_val": "123.456",
            "bool_str": "true",
            "null_str": "null",
            "empty": "",
            "complex": "### Header\n**Field**: Value"
        }
    ]
    
    print("=" * 80)
    print("OPTION 1: Formatted Text (Human Readable)")
    print("=" * 80)
    
    # Create document collection with formatted text
    doc_collection_text = create_document_from_list(
        data_list=data_list,
        document_id="rag-doc",
        document_url="rag.json",
        document_version="1.0"
    )
    
    print(doc_collection_text.to_json())
    print("\n")
    
    print("=" * 80)
    print("OPTION 2: JSON Format (Machine Readable)")
    print("=" * 80)
    
    # Create document collection preserving JSON
    doc_collection_json = create_document_from_list_json(
        data_list=data_list,
        document_id="rag-doc",
        document_url="rag.json",
        document_version="1.0",
        preserve_json=True
    )
    
    print(doc_collection_json.to_json())
    print("\n")
    
    print("=" * 80)
    print("CONVERSION TO LANGCHAIN")
    print("=" * 80)
    
    # Convert to LangChain documents
    langchain_docs = doc_collection_json.to_langchain_docs()
    
    print(f"Total LangChain Documents: {len(langchain_docs)}\n")
    
    for i, doc in enumerate(langchain_docs, 1):
        print(f"Document {i}:")
        print(f"  Content Preview: {doc.page_content[:100]}...")
        print(f"  Metadata: {doc.metadata}")
        print()
    
    print("=" * 80)
    print("ACCESSING SPECIFIC CHUNKS")
    print("=" * 80)
    
    # Access specific chunk
    chunk_2 = doc_collection_json.items[1]
    print(f"Chunk 2 Content:\n{chunk_2.chunkTokens}\n")
    print(f"Chunk 2 Metadata:")
    print(f"  Document ID: {chunk_2.metadata.documentId}")
    print(f"  Sequence: {chunk_2.metadata.chunkSequence}/{chunk_2.metadata.chunkTotal}")
    print(f"  Last Modified: {chunk_2.metadata.lastModified}")
    
    print("\n" + "=" * 80)
    print("SAVE TO FILE")
    print("=" * 80)
    
    # Save to file
    doc_collection_json.to_json("output_rag.json")
    print("✓ Saved to output_rag.json")
    
    # Load from file
    loaded = CustomDocumentCollection.from_json("output_rag.json")
    print(f"✓ Loaded {len(loaded.items)} chunks from file")

OPTION 1: Formatted Text (Human Readable)
{
  "items": [
    {
      "chunkTokens": "rule: fnd test\nsql: BAT.FIND_TEST_IND = TRUE\ncategory: FIND_TEST_IND\nbat_col_meta_info:\n  ### FIND_TEST_IND\n  **Attribute Name**: 'FIND_TEST_IND' \n  **Data Types**: BOOLEAN\n  **Description**: A bool field that indicates whether or not an account was part of Find Test. If 'Null', then the account was not part of a FT. ",
      "metadata": {
        "documentId": "rag-doc",
        "documentUrl": "rag.json",
        "documentVersion": "1.0",
        "chunkSequence": 1,
        "chunkTotal": 4,
        "lastModified": "2025-12-29T16:16:30.602554"
      }
    },
    {
      "chunkTokens": "id: 123\nname: Test User\nactive: True\nscore: 95.5",
      "metadata": {
        "documentId": "rag-doc",
        "documentUrl": "rag.json",
        "documentVersion": "1.0",
        "chunkSequence": 2,
        "chunkTotal": 4,
        "lastModified": "2025-12-29T16:16:30.602588"
      }
    },
    {
      "chunk

/var/folders/41/1w73md7s4dl39qztvt7zf8800000gn/T/ipykernel_98959/2145891994.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  lastModified: str = Field(default_factory=lambda: datetime.utcnow().isoformat())
